In [ ]:
!unzip 'data.zip'     # unzipping training data along with cross validation data

In [ ]:
# Importing the libraries
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
# Part 1 - Data Preprocessing
# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
                                   )
training_set = train_datagen.flow_from_directory('/content/data/Train Data',
                                                 target_size = (200, 200),
                                                 batch_size = 32,
                                                 class_mode = 'binary')
# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/data/Test Data',
                                            target_size = (200, 200),
                                            batch_size = 32,
                                            class_mode = 'binary')

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, BatchNormalization

# Building the  Convolutional Neural Network
classifier = Sequential([
             Convolution2D(64, 3, 3, input_shape = (200, 200, 3), activation = 'relu'),
             MaxPooling2D(pool_size = (2, 2)),
             Convolution2D(32, 3, 3, activation = 'relu'),
             MaxPooling2D(pool_size = (2, 2)),
             Flatten(),
             Dense( 128, activation = 'relu'),
             Dense( 64, activation = 'relu'),
             Dense( 1, activation = 'sigmoid')])


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

In [ ]:
history = classifier.fit(x = training_set, validation_data = test_set, epochs = 25)

In [ ]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = classifier.evaluate(test_set)

In [ ]:
import pandas as pd
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
!unzip 'TestSet.zip'

In [ ]:
from os import listdir
from PIL import Image as PImage
import numpy as np
def loadImages(path):
    # return array of images

    imagesList = listdir(path)
    y_pred = []
    for image in imagesList:
        pathimg = path + image
        test_image = tf.keras.utils.load_img(pathimg, target_size = (200, 200))
        test_image = tf.keras.utils.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = classifier.predict(test_image)
        training_set.class_indices
        if result[0][0] == 1:
          prediction = 1
        else:
          prediction = 0
        y_pred.append(prediction)
    return y_pred


In [ ]:
path = '/content/TestSet/'
y_pred = loadImages(path)

In [ ]:
for i in range(20):
  print(y_pred[i])